### Load Dataset

In [ ]:
[i['inputs'][j] for j in i['inputs'] for i in [{'amit':234, 'inputs':234},{'garg':234, 'inputs':98}]]

In [2]:
import pandas as pd
import numpy as np

In [ ]:
from datasets import load_dataset

df = load_dataset("csv", "multinerd_en.tsv", sep="\t", encoding='utf-8')

In [103]:
import re
re.split(pattern=r"[ \t ]", string="asdakjhhd asdjkhakjd \tsdfj")

['asdakjhhd', 'asdjkhakjd', '', 'sdfj']

In [1]:
from omegaconf import OmegaConf
YAML_CONFIG = OmegaConf.load('./config.yaml')

In [2]:
def get_scatter_configs():
    """Get scatter config details that includes silo's compute, datastore, inputs, etc."""

    scatter_configs = []
    for silo_config in YAML_CONFIG.strategy.horizontal:
        scatter_config = {}
        scatter_config["inputs"] = {
            "raw_train_data": Input(
                type=silo_config.inputs.raw_training_data.type,
                mode=silo_config.inputs.raw_training_data.mode,
                path=silo_config.inputs.raw_training_data.path,
            ),
            "raw_test_data": Input(
                type=silo_config.inputs.raw_testing_data.type,
                mode=silo_config.inputs.raw_testing_data.mode,
                path=silo_config.inputs.raw_testing_data.path,
            ),
        }
        scatter_config["computes"] = silo_config["computes"]
        scatter_config["datastore"] = silo_config["datastore"]
        scatter_config["name"] = silo_config["name"]
        scatter_configs.append(scatter_config)

    return scatter_configs

In [4]:
from azure.ai.ml import MLClient, Input
get_scatter_configs()

[{'inputs': {'raw_train_data': {'type': 'uri_file', 'path': 'https://azureopendatastorage.blob.core.windows.net/mnist/processed/train.csv', 'mode': 'download'},
   'raw_test_data': {'type': 'uri_file', 'path': 'https://azureopendatastorage.blob.core.windows.net/mnist/processed/t10k.csv', 'mode': 'download'}},
  'computes': ['silo0-01', 'silo0-02'],
  'datastore': 'datastore_silo0',
  'name': 'silo0'},
 {'inputs': {'raw_train_data': {'type': 'uri_file', 'path': 'https://azureopendatastorage.blob.core.windows.net/mnist/processed/train.csv', 'mode': 'download'},
   'raw_test_data': {'type': 'uri_file', 'path': 'https://azureopendatastorage.blob.core.windows.net/mnist/processed/t10k.csv', 'mode': 'download'}},
  'computes': ['silo1-01', 'silo1-02'],
  'datastore': 'datastore_silo1',
  'name': 'silo1'},
 {'inputs': {'raw_train_data': {'type': 'uri_file', 'path': 'https://azureopendatastorage.blob.core.windows.net/mnist/processed/train.csv', 'mode': 'download'},
   'raw_test_data': {'type': 

In [8]:
# [ silo_config["inputs"]["raw_training_data"]=Input(**silo_config["inputs"]["raw_training_data"])  for silo_config in YAML_CONFIG.strategy.horizontal if True] 

{silo_config[conf]["raw_training_data"]: Input(**silo_config["inputs"]["raw_training_data"]) if conf=="inputs" else conf: silo_config[conf] for silo_config in YAML_CONFIG.strategy.horizontal for conf in silo_config }

SyntaxError: invalid syntax (2247262650.py, line 3)

In [14]:
final_conf = []
for silo_config in YAML_CONFIG.strategy.horizontal:
    tmp = {}
    for conf in silo_config:
        tmp[conf] = silo_config[conf]
        if conf=="inputs":
            for raw_key in silo_config[conf]:
                print(tmp[conf][raw_key])
                tmp[conf][raw_key] = Input(**silo_config[conf][raw_key])
    final_conf.append(tmp)
final_conf

{'type': 'uri_file', 'mode': 'download', 'path': 'https://azureopendatastorage.blob.core.windows.net/mnist/processed/train.csv'}


UnsupportedValueType: Value 'Input' is not a supported primitive type
    full_key: strategy.horizontal[0].inputs.raw_training_data
    object_type=dict

In [ ]:
from sklearn.model_selection import train_test_split


def preprocess_data(file_name):
    data = pd.read_csv(file_name, sep='\t', header=None)
    labels = data.iloc[:, 2].value_counts().iloc[:32].index
    filtered_data = data.loc[:, 0:2].values.tolist()
    labelsToId = {label: idx for idx, label in enumerate(labels)}
    processed_data =[]
    ner_tag = []
    sentence=[]
    for line in filtered_data:
        if int(line[0])==0:
            if len(ner_tag)>0:
                processed_data.append([sentence, ner_tag])
            ner_tag =[]
            sentence=[]
        sentence.append(str(line[1]).strip())
        ner_tag.append(labelsToId.get(str(line[2]).strip(), -1))
    filtered_dt = list(filter(lambda val: False if -1 in val[1] else True, processed_data))
    final_df = pd.DataFrame(filtered_dt, columns=["sentence", "label"])
    train_set, val_set = train_test_split(final_df, test_size=0.1)
    train_set = train_set.reset_index(drop=True)
    val_set = val_set.reset_index(drop=True)
    return train_set, val_set

In [188]:
data = pd.read_csv('multinerd_en.tsv', sep='\t', header=None)

In [2]:
from datasets import load_dataset

df_ner = load_dataset("tner/multinerd", 'en')

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 197.82it/s]


Dataset multinerd downloaded and prepared to C:/Users/Amit Garg/.cache/huggingface/datasets/tner___multinerd/en/1.0.0/93d8b23b97a50218e8fe28193b46854b1e50f260e1d1c174d71202d2ed29e9a0. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 41.67it/s]


In [3]:
df_ner

DatasetDict({
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 164144
    })
})

In [4]:
df_ner['test'][0]

{'tokens': ['Created',
  'by',
  'James',
  'Cameron',
  'and',
  'Charles',
  'H.',
  'Eglee',
  ',',
  'it',
  'stars',
  'Jessica',
  'Alba',
  'in',
  'her',
  'breakthrough',
  'role',
  '.'],
 'tags': [0, 0, 1, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0]}

In [189]:
# with open("multinerd_en.tsv", "r", encoding='utf-8') as f:
#     lines = f.read().split("\n")
# data=[]
# for idx, line in enumerate(lines):
#     # print(line)
#     vals = re.split(pattern=r"[\t ]", string=line)
#     if len(vals)<3:
#         continue
#     if int(vals[0].strip())==0:
#         idx +=1
#     token = vals[1]
#     label = vals[2]
#     data.append((idx, token, label))

In [190]:
data

,0,1,2,3,4,5,6,7,8,9
0,0,Created,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,by,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,James,B-PER,bn:00786804n,Q42574,15622.0,James_Cameron,James Francis Cameron is a Canadian film direc...,https://upload.wikimedia.org/wikipedia/commons...,NaN
3,3,Cameron,I-PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,and,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2397390,30,fifteen,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2397391,31,members,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2397392,32,this,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2397393,33,afternoon,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
labels = data.iloc[:, 2].value_counts().iloc[:32].index

In [192]:
idToLabel = {idx: label for idx, label in enumerate(labels)}

In [193]:
labelsToId = {label: idx for idx, label in enumerate(labels)}

In [194]:
labelsToId

{'O': 0,
 'I-PER': 1,
 'B-PER': 2,
 'B-LOC': 3,
 'I-ORG': 4,
 'B-ORG': 5,
 'I-LOC': 6,
 'B-ANIM': 7,
 'B-DIS': 8,
 'B-FOOD': 9,
 'I-MEDIA': 10,
 'B-PLANT': 11,
 'I-DIS': 12,
 'I-ANIM': 13,
 'I-EVE': 14,
 'B-MEDIA': 15,
 'I-FOOD': 16,
 'B-EVE': 17,
 'B-TIME': 18,
 'B-CEL': 19,
 'I-PLANT': 20,
 'I-TIME': 21,
 'I-CEL': 22,
 'B-SUPER': 23,
 'I-VEHI': 24,
 'B-VEHI': 25,
 'B-INST': 26,
 'I-INST': 27,
 'I-PHY': 28,
 'I-SUPER': 29,
 'B-BIO': 30,
 'B-PHY': 31}

In [195]:
idToLabel

{0: 'O',
 1: 'I-PER',
 2: 'B-PER',
 3: 'B-LOC',
 4: 'I-ORG',
 5: 'B-ORG',
 6: 'I-LOC',
 7: 'B-ANIM',
 8: 'B-DIS',
 9: 'B-FOOD',
 10: 'I-MEDIA',
 11: 'B-PLANT',
 12: 'I-DIS',
 13: 'I-ANIM',
 14: 'I-EVE',
 15: 'B-MEDIA',
 16: 'I-FOOD',
 17: 'B-EVE',
 18: 'B-TIME',
 19: 'B-CEL',
 20: 'I-PLANT',
 21: 'I-TIME',
 22: 'I-CEL',
 23: 'B-SUPER',
 24: 'I-VEHI',
 25: 'B-VEHI',
 26: 'B-INST',
 27: 'I-INST',
 28: 'I-PHY',
 29: 'I-SUPER',
 30: 'B-BIO',
 31: 'B-PHY'}

In [196]:
filtered_data = data.loc[:, 0:2].values.tolist()

In [197]:
filtered_data

[[0, 'Created', 'O'],
 [1, 'by', 'O'],
 [2, 'James', 'B-PER'],
 [3, 'Cameron', 'I-PER'],
 [4, 'and', 'O'],
 [5, 'Charles', 'B-PER'],
 [6, 'H.', 'I-PER'],
 [7, 'Eglee', 'I-PER'],
 [8, ',', 'O'],
 [9, 'it', 'O'],
 [10, 'stars', 'O'],
 [11, 'Jessica', 'B-PER'],
 [12, 'Alba', 'I-PER'],
 [13, 'in', 'O'],
 [14, 'her', 'O'],
 [15, 'breakthrough', 'O'],
 [16, 'role', 'O'],
 [17, '.', 'O'],
 [0, 'In', 'O'],
 [1, 'a', 'O'],
 [2, 'post-apocalyptic', 'O'],
 [3, 'Seattle', 'B-LOC'],
 [4, ',', 'O'],
 [5, 'she', 'O'],
 [6, 'tries', 'O'],
 [7, 'to', 'O'],
 [8, 'lead', 'O'],
 [9, 'a', 'O'],
 [10, 'normal', 'O'],
 [11, 'life', 'O'],
 [12, 'while', 'O'],
 [13, 'eluding', 'O'],
 [14, 'capture', 'O'],
 [15, 'by', 'O'],
 [16, 'government', 'O'],
 [17, 'agents', 'O'],
 [18, 'and', 'O'],
 [19, 'searching', 'O'],
 [20, 'for', 'O'],
 [21, 'her', 'O'],
 [22, 'brothers', 'O'],
 [23, 'and', 'O'],
 [24, 'sisters', 'O'],
 [25, 'scattered', 'O'],
 [26, 'in', 'O'],
 [27, 'the', 'O'],
 [28, 'aftermath', 'O'],
 [29, 'of

In [199]:
processed_data =[]
ner_tag = []
sentence=[]
for line in filtered_data:
    if int(line[0])==0:
        if len(ner_tag)>0:
            processed_data.append([sentence, ner_tag])
        ner_tag =[]
        sentence=[]
    sentence.append(str(line[1]).strip())
    ner_tag.append(labelsToId.get(str(line[2]).strip(), -1))


In [200]:
len(processed_data)

113008

In [201]:
processed_data[0]

[['Created',
  'by',
  'James',
  'Cameron',
  'and',
  'Charles',
  'H.',
  'Eglee',
  ',',
  'it',
  'stars',
  'Jessica',
  'Alba',
  'in',
  'her',
  'breakthrough',
  'role',
  '.'],
 [0, 0, 2, 1, 0, 2, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0]]

In [202]:
labels

Index(['O', 'I-PER', 'B-PER', 'B-LOC', 'I-ORG', 'B-ORG', 'I-LOC', 'B-ANIM',
       'B-DIS', 'B-FOOD', 'I-MEDIA', 'B-PLANT', 'I-DIS', 'I-ANIM', 'I-EVE',
       'B-MEDIA', 'I-FOOD', 'B-EVE', 'B-TIME', 'B-CEL', 'I-PLANT', 'I-TIME',
       'I-CEL', 'B-SUPER', 'I-VEHI', 'B-VEHI', 'B-INST', 'I-INST', 'I-PHY',
       'I-SUPER', 'B-BIO', 'B-PHY'],
      dtype='object')

In [203]:
filtered_dt = list(filter(lambda val: False if -1 in val[1] else True, processed_data))

In [204]:
len(filtered_dt)

111634

In [205]:
filtered_dt

[[['Created',
   'by',
   'James',
   'Cameron',
   'and',
   'Charles',
   'H.',
   'Eglee',
   ',',
   'it',
   'stars',
   'Jessica',
   'Alba',
   'in',
   'her',
   'breakthrough',
   'role',
   '.'],
  [0, 0, 2, 1, 0, 2, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0]],
 [['In',
   'a',
   'post-apocalyptic',
   'Seattle',
   ',',
   'she',
   'tries',
   'to',
   'lead',
   'a',
   'normal',
   'life',
   'while',
   'eluding',
   'capture',
   'by',
   'government',
   'agents',
   'and',
   'searching',
   'for',
   'her',
   'brothers',
   'and',
   'sisters',
   'scattered',
   'in',
   'the',
   'aftermath',
   'of',
   'their',
   'escape',
   '.'],
  [0,
   0,
   0,
   3,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]],
 [['The',
   'first',
   'episode',
   'was',
   'behind',
   'only',
   'O\t\t\t\t\t\t\t\n7\tCSI\tB-MEDIA\tbn:01823499n\tQ117396\t247787\tCS

In [208]:
final_df = pd.DataFrame(filtered_dt, columns=["sentence", "label"])

In [209]:
final_df

,sentence,label
0,"[Created, by, James, Cameron, and, Charles, H....","[0, 0, 2, 1, 0, 2, 1, 1, 0, 0, 0, 2, 1, 0, 0, ..."
1,"[In, a, post-apocalyptic, Seattle, ,, she, tri...","[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[The, first, episode, was, behind, only, O\t\t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[The, history, of, anarchism, goes, back, to, ...","[0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0,..."
4,"[Anarchists, have, taken, part, in, several, r...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 14, 1..."
...,...,...
111629,"[Urban, growth, is, a, particular, concern, in...","[0, 0, 0, 0, 0, 0, 0, 3, 6, 0]"
111630,"[Pakistan, ,, Saudi, Arabia, and, Russia, were...","[3, 0, 3, 6, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
111631,"[The, protesters, are, calling, for, an, end, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0]"
111632,[O\t\t\t\t\t\t\t\n1\tThe\tO\t\t\t\t\t\t\t\n2\t...,"[0, 0, 0, 0, 0, 0, 0, 3, 6, 0, 0, 0]"


In [253]:
from sklearn.model_selection import train_test_split

train_set, val_set = train_test_split(final_df, test_size=0.1)

In [254]:
len(train_set), len(val_set)

(100470, 11164)

In [262]:
train_set = train_set.reset_index(drop=True)
val_set = val_set.reset_index(drop=True)

In [263]:
from datasets import Dataset, DatasetDict

tds = Dataset.from_pandas(train_set)
vds = Dataset.from_pandas(val_set)
ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds

In [264]:
ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100470
    })
    validation: Dataset({
        features: ['sentence', 'label'],
        num_rows: 11164
    })
})

In [265]:
ds["train"][0]

{'sentence': ['The',
  'middle',
  'stretches',
  'of',
  'the',
  'river',
  'have',
  'many',
  'limestone',
  'caves',
  'which',
  'were',
  'inhabited',
  'by',
  'humans',
  'in',
  'the',
  'prehistoric',
  'era',
  ';',
  'the',
  'caves',
  'are',
  'several',
  'types',
  'of',
  'limestone',
  'formations',
  ',',
  'namely',
  'tuffeau',
  '(',
  'a',
  'porous',
  'type',
  'of',
  'chalk',
  ',',
  'not',
  'to',
  'be',
  'confused',
  'with',
  'tufa',
  ')',
  'and',
  'Falun',
  '(',
  'formed',
  '12',
  'O\t\t\t\t\t\t\t\n51\tmillion\tO\t\t\t\t\t\t\t\n52\tyears\tO\t\t\t\t\t\t\t\n53\tago\tO\t\t\t\t\t\t\t\n54\t)\tO\t\t\t\t\t\t\t\n55\t.\tO\t\t\t\t\t\t\t\n\n0\tThe\tO\t\t\t\t\t\t\t\n1\tfinal\tO\t\t\t\t\t\t\t\n2\tdam\tO\t\t\t\t\t\t\t\n3\twas\tO\t\t\t\t\t\t\t\n4\tdecommissioned\tO\t\t\t\t\t\t\t\n5\tby\tO\t\t\t\t\t\t\t\n6\tÉlectricité\tB-ORG\tbn:00118158n\tQ274591\t1000559\tÉlectricité_de_France\tÉlectricité de France S.A., commonly known as EDF, is a French multinational el

In [266]:
ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100470
    })
    validation: Dataset({
        features: ['sentence', 'label'],
        num_rows: 11164
    })
})

## Preprocess Data

In [1]:
from datasets import load_dataset

df_ner = load_dataset("tner/multinerd", 'en', split='test')

Found cached dataset multinerd (C:/Users/Amit Garg/.cache/huggingface/datasets/tner___multinerd/en/1.0.0/93d8b23b97a50218e8fe28193b46854b1e50f260e1d1c174d71202d2ed29e9a0)


In [23]:
df_ner = df_ner.shuffle(seed=42)
tmp = df_ner.shard(num_shards=3, index=0, contiguous=False)
tmp1 = df_ner.shard(num_shards=3, index=1, contiguous=False)
tmp2 = df_ner.shard(num_shards=3, index=2, contiguous=False)

In [2]:
ds = df_ner.train_test_split(test_size=0.1)

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 147729
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 16415
    })
})

In [27]:
from datasets import load_dataset
from transformers import AutoTokenizer

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

def preprocess_data(ds):
    return ds.map(
        tokenize_and_align_labels,
        batched=True,
        remove_columns=ds["train"].column_names,
    )

In [28]:

df_ner = load_dataset("tner/multinerd", 'en', split='test')
ds = df_ner.train_test_split(test_size=0.1)

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenized_datasets = preprocess_data(ds)

Found cached dataset multinerd (C:/Users/Amit Garg/.cache/huggingface/datasets/tner___multinerd/en/1.0.0/93d8b23b97a50218e8fe28193b46854b1e50f260e1d1c174d71202d2ed29e9a0)
 94%|█████████▍| 16/17 [00:01<00:00,  8.60ba/s]


In [29]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 147729
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 16415
    })
})

In [27]:
tokenized_datasets["train"].shape[0]

147729

In [6]:
tokenized_datasets["train"].save_to_disk("./training_data")

In [10]:
import datasets
tmp = datasets.load_from_disk("./training_data/")

### Archive

In [8]:
df_ner

DatasetDict({
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 164144
    })
})

In [7]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenized_datasets = ds.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=ds["train"].column_names,
)

In [268]:
tokenizer.is_fast

True

In [11]:
df_ner["test"][0]["tokens"]

['Created',
 'by',
 'James',
 'Cameron',
 'and',
 'Charles',
 'H.',
 'Eglee',
 ',',
 'it',
 'stars',
 'Jessica',
 'Alba',
 'in',
 'her',
 'breakthrough',
 'role',
 '.']

In [14]:
inputs = tokenizer(df_ner["test"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'Created',
 'by',
 'James',
 'Cameron',
 'and',
 'Charles',
 'H',
 '.',
 'E',
 '##gle',
 '##e',
 ',',
 'it',
 'stars',
 'Jessica',
 'Alba',
 'in',
 'her',
 'breakthrough',
 'role',
 '.',
 '[SEP]']

In [15]:
inputs.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 6,
 7,
 7,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 None]

In [19]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [21]:
labels = df_ner["test"][0]["tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 1, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0]
[-100, 0, 0, 1, 2, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, -100]


In [273]:
def tokenize_and_align_labels(examples):
    try:
        tokenized_inputs = tokenizer(
            examples["sentence"], truncation=True, is_split_into_words=True
        )
        all_labels = examples["label"]
        new_labels = []
        for i, labels in enumerate(all_labels):
            word_ids = tokenized_inputs.word_ids(i)
            new_labels.append(align_labels_with_tokens(labels, word_ids))

        tokenized_inputs["labels"] = new_labels
    except:
        print(examples)
    return tokenized_inputs

In [274]:
ds["train"].column_names

['sentence', 'label']

In [275]:
tokenized_datasets = ds.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=ds["train"].column_names,
)

 92%|█████████▏| 11/12 [00:01<00:00,  7.55ba/s]


In [276]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 100470
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 11164
    })
})

## Modeling

In [30]:
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, AutoTokenizer
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
import pandas as pd
import evaluate
import numpy as np
from tqdm.auto import tqdm
import torch

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = evaluate.load("seqeval")

labelToId = pd.read_json(r'C:\Users\Amit Garg\OneDrive - Microsoft\Documents\azure-samples-main-repo\azure-ml-federated-learning\examples\components\PrivateEntityRecognition\traininsilo\labels.json', typ='series').to_dict()
idToLabel = {val: key for key, val in labelToId.items()}

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[idToLabel[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [idToLabel[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=idToLabel,
    label2id=labelToId,
)


train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=8
)


optimizer = AdamW(model.parameters(), lr=2e-5)


num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[idToLabel[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [idToLabel[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions


    

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        # print(predictions)
        # print(labels)

        true_predictions, true_labels = postprocess(predictions, labels)
        metric.add_batch(predictions=true_predictions, references=true_labels)

        results = metric.compute()
        print(
            f"epoch {epoch}:",
            {
                key: results[f"overall_{key}"]
                for key in ["precision", "recall", "f1", "accuracy"]
            },
        )
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


for epoch in range(self._epochs):

    running_loss = 0.0
    num_of_batches_before_logging = 100
    # Training
    self.model_.train()
    for i, batch in enumerate(self.train_loader_):
        outputs = self.model_(**batch)
        loss = outputs.loss

        self.optimizer_.step()
        lr_scheduler.step()
        self.optimizer_.zero_grad()
        progress_bar.update(1)

        running_loss += loss.item() / len(batch)
        if i != 0 and i % num_of_batches_before_logging == 0:
            training_loss = running_loss / num_of_batches_before_logging
            logger.info(
                f"Epoch: {epoch}/{self._epochs}, Iteration: {i}, Training Loss: {training_loss}"
            )

            # log train loss
            self.log_metrics(
                mlflow_client,
                root_run_id,
                "Train Loss",
                training_loss,
            )

            # log accuracy
            self.log_metrics(
                mlflow_client,
                root_run_id,
                "Train accuracy",
                training_loss,
            )

            # log f1 score
            self.log_metrics(
                mlflow_client,
                root_run_id,
                "Train f1-score",
                training_loss,
            )


            # log precision
            self.log_metrics(
                mlflow_client,
                root_run_id,
                "Train precision",
                training_loss,
            )


            # log recall
            self.log_metrics(
                mlflow_client,
                root_run_id,
                "Train recall",
                training_loss,
            )

            running_loss = 0.0

In [32]:
tokenized_datasets["test"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 16415
})

In [65]:
!pip install accelerate

In [61]:
pred.logits.detach().cpu().clone().numpy().shape

(8, 45, 35)

In [72]:
len(train_dataloader)

18467

In [75]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [76]:
device

device(type='cpu')

In [82]:
for item in train_dataloader:
    item = {key: value.to(device) for key, value in item.items()}
    # item["input_ids"] = item["input_ids"].to(device)
    print(item.keys())
    break 

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
for item in train_dataloader:
    # pred = model(**item)
    

    # print(f"Predicted: {pred.logits.shape}")
    print(f"Actual: {item['labels'].shape}")
    print(f"Actual: {item['input_ids'].shape}")

In [69]:
for item in eval_dataloader:
    pred = model(**item)
    print(f"Predicted: {pred.logits.shape}")
    print(f"Actual: {item['labels'].shape}")
    true_predictions, true_labels = postprocess( pred.logits.argmax(dim=-1), item['labels'])
    print(true_predictions)
    print(true_labels)
    metric.add_batch(predictions=true_predictions, references=true_labels)
    break

Predicted: torch.Size([8, 52, 35])
Actual: torch.Size([8, 52])
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O'], ['O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FOOD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'I-LOC', 'O'], ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [70]:
results = metric.compute()
print(
    f"epoch {1}:",
    {
        key: results[f"overall_{key}"]
        for key in ["precision", "recall", "f1", "accuracy"]
    },
)

epoch 1: {'precision': 0.1, 'recall': 0.006369426751592357, 'f1': 0.011976047904191617, 'accuracy': 0.015789473684210527}


c:\Users\Amit Garg\miniconda3\envs\fl-v2-dev\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
pred.logits.argmax(dim=-1)

tensor([[21,  8, 20,  4, 25,  2, 11, 14, 29, 11,  2, 34, 16, 16, 25, 20, 18, 20,
         25, 25, 25, 14, 25, 14, 25, 18,  2,  2, 18, 14, 25, 25, 20, 25, 25, 20,
         25, 25, 14, 14, 25, 18,  2, 18, 18, 18, 20, 20, 18, 20, 25, 25],
        [21,  8, 20, 28, 20, 14, 14,  4, 28, 20, 12, 20, 20, 16, 16, 20, 20, 18,
         18, 29, 29, 29, 14, 18, 18, 14, 20, 20, 20, 20, 20, 29, 20, 20, 29,  2,
         29, 12, 12, 18, 18, 18, 12, 20, 20, 20, 20, 20, 20, 20, 29, 29],
        [14, 21, 30, 20,  4, 30,  4,  3, 20,  4,  2,  3, 30, 30, 24, 27, 34, 19,
         10, 14, 13, 28, 14, 32, 12,  2, 34, 16, 16,  4,  4,  4,  4, 29,  3,  2,
          2, 14,  4,  2,  2,  2,  2,  2,  4,  4,  4,  4,  4,  4, 14, 14],
        [34,  8, 14,  2, 20, 20, 20, 19, 19, 20, 20, 20,  4, 34,  4,  3, 20,  2,
         12, 12, 20, 34,  4, 12, 28, 30, 20, 20,  4, 12, 12, 20, 30, 30,  2,  2,
          2,  2,  2,  2,  2, 20, 20,  4,  4,  4,  4, 20,  4,  2,  4,  4],
        [34, 21,  6, 20,  3, 27,  4, 17, 14, 27, 11, 11,

In [ ]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        # print(predictions)
        # print(labels)

        true_predictions, true_labels = postprocess(predictions, labels)
        metric.add_batch(predictions=true_predictions, references=true_labels)

        results = metric.compute()
        print(
            f"epoch {epoch}:",
            {
                key: results[f"overall_{key}"]
                for key in ["precision", "recall", "f1", "accuracy"]
            },
        )
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [15]:
import torch
torch.save(model.state_dict(), './model_dict.pt')
model.load_state_dict(torch.load("./model_dict.pt"))

<All keys matched successfully>

In [277]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [13]:
import pandas as pd
labelToId = pd.read_json(r'C:\Users\Amit Garg\OneDrive - Microsoft\Documents\azure-samples-main-repo\azure-ml-federated-learning\examples\components\PrivateEntityRecognition\preprocessing\config\labels.json', typ='series').to_dict()

In [16]:
idToLabel = {val: key for key, val in labelToId.items()}

In [278]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    3,    3,    3,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [279]:
!pip install seqeval

In [280]:
import evaluate

metric = evaluate.load("seqeval")

In [240]:
!pip install evaluate

     ---------------------------------------- 72.9/72.9 kB 2.0 MB/s eta 0:00:00


In [281]:
metric

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [282]:
labels = ds["train"][0]["label"]
labels = [idToLabel[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [283]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [284]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[idToLabel[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [idToLabel[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [285]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=idToLabel,
    label2id=labelsToId,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [286]:
model.config.num_labels

32

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[idToLabel[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [idToLabel[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        # accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [287]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [288]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [294]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [295]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[idToLabel[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [idToLabel[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [296]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        # accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # # Evaluation
    # model.eval()
    # for batch in eval_dataloader:
    #     with torch.no_grad():
    #         outputs = model(**batch)

    #     predictions = outputs.logits.argmax(dim=-1)
    #     labels = batch["labels"]

    #     # Necessary to pad predictions and labels for being gathered
    #     predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
    #     labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

    #     predictions_gathered = accelerator.gather(predictions)
    #     labels_gathered = accelerator.gather(labels)

    #     true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
    #     metric.add_batch(predictions=true_predictions, references=true_labels)

    # results = metric.compute()
    # print(
    #     f"epoch {epoch}:",
    #     {
    #         key: results[f"overall_{key}"]
    #         for key in ["precision", "recall", "f1", "accuracy"]
    #     },
    # )

 30%|███       | 11454/37677 [2:38:52<18:14:59,  2.51s/it]  

## Registry Bug Repro

In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.entities import (
    Environment,
    BuildContext,
    Model,
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    CodeConfiguration,
)
from azure.ai.ml.constants import AssetTypes
import time, datetime, os

# print the sdk version - you many want to share this in the issue you will report if parts of this notebook don't work
!pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.2.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: c:\users\amit garg\miniconda3\envs\fl_experiment_conda_env\lib\site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 


In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [4]:
ml_client_registry = MLClient(credential=credential, registry_name="aims-bugbash")
print(ml_client_registry)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x00000235FE8B76A0>,
         subscription_id=48bbc269-ce89-4f6f-9a12-c6f91fcb772d,
         resource_group_name=aml1p-rg,
         workspace_name=None)


In [10]:
train_component_from_registry = ml_client_registry.components.get(
    name="data_pipeline", version="3.1415"
)
print(train_component_from_registry)

$schema: https://azuremlschemas.azureedge.net/latest/pipelineComponent.schema.json
name: data_pipeline
version: '3.1415'
display_name: pipeline component with data prep and transformation
description: pipeline component with data prep and transformation
type: pipeline
inputs:
  raw_data:
    type: uri_folder
outputs:
  train_data:
    type: uri_folder
id: azureml://registries/aims-bugbash/components/data_pipeline/versions/3.1415
is_deterministic: false
creation_context:
  created_at: '2022-11-30T00:27:16.046177+00:00'
  created_by: Thomas Portet
  created_by_type: User
  last_modified_at: '2022-11-30T00:27:16.046177+00:00'
  last_modified_by: Thomas Portet
  last_modified_by_type: User



### Multiply dataset

In [2]:
import os
import argparse
import logging
import glob
import pathlib
import shutil
from tqdm import tqdm

In [4]:
all_files_list = list(glob.glob(os.path.join("C:/Users/Amit Garg/OneDrive - Microsoft/Documents/azure-samples-main-repo/azure-ml-federated-learning", "**", "*"), recursive=True))

In [6]:
len(all_files_list)

170

In [7]:
def multiply_files(file_paths, source, target, multiplication_factor, increment=0):
    """Multiply a given folder."""
    files_created = 0

    for i in range(multiplication_factor):
        for entry in tqdm(file_paths):
            if os.path.isfile(entry):
                # get name of file
                file_name = pathlib.Path(os.path.basename(entry))

                # get path to the file
                full_input_dir = os.path.dirname(entry)

                # create path to the output
                rel_dir = os.path.relpath(full_input_dir, source)
                full_output_dir = os.path.join(target, rel_dir)

                # create a name for the output file
                output_file_path = os.path.join(full_output_dir, file_name.stem + f"_{i+increment}" + file_name.suffix)

                # create output dir
                if not os.path.isdir(full_output_dir):
                    logging.getLogger(__name__).info(f"Creating output subfolder {full_output_dir}")
                    os.makedirs(full_output_dir, exist_ok=True)

                if not os.path.isfile(output_file_path):
                    shutil.copy(entry, output_file_path)
                    files_created += 1

        logging.getLogger(__name__).info(f"Achieved multication {i}")

    return files_created

In [8]:
multiply_files(all_files_list, "C:/Users/Amit Garg/OneDrive - Microsoft/Documents/azure-samples-main-repo/azure-ml-federated-learning", "C:/Users/Amit Garg/OneDrive - Microsoft/Documents/AzureML", 3)

100%|██████████| 170/170 [00:00<00:00, 1057.74it/s]


366

In [8]:
set([i for _silo in [["1","2"],["3","4"]] for i in _silo])

{'1', '2', '3', '4'}

### NER ERROR DEBUG

In [1]:
import argparse
import logging
import sys
import os

from transformers import (
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    AutoTokenizer,
    get_scheduler,
)
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
from datasets import load_from_disk
from torch.optim import AdamW
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import evaluate
import mlflow
import torch
from distutils.util import strtobool

# DP
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator

c:\Users\Amit Garg\miniconda3\envs\fl_experiment_conda_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class NERTrainer:
    def __init__(
        self,
        tokenizer_name,
        model_name,
        train_data_dir="./",
        test_data_dir="./",
        model_path=None,
        lr=0.01,
        epochs=1,
        batch_size=64,
        dp=False,
        dp_target_epsilon=50.0,
        dp_target_delta=1e-5,
        dp_max_grad_norm=1.0,
        experiment_name="default-experiment",
        iteration_num=1,
        total_num_of_iterations=1,
        device_id=None,
        distributed=False,
    ):
        """NER Trainer trains BERT-base model (default) on the MultiNERD dataset.

        Args:
            tokenizer_name(str): Tokenizer name.
            model_name(str): Model name.
            train_data_dir(str, optional): Training data directory path.
            test_data_dir(str, optional): Testing data directory path.
            model_path (str, optional): Model path. Defaults to None.
            lr (float, optional): Learning rate. Defaults to 0.01.
            epochs (int, optional): number of epochs. Defaults to 1.
            batch_size (int, optional): DataLoader batch size. Defaults to 64.
            dp (bool, optional): Differential Privacy. Default is False
            dp_target_epsilon (float, optional): DP target epsilon. Default is 50.0
            dp_target_delta (float, optional): DP target delta. Default is 1e-5
            dp_max_grad_norm (float, optional): DP max gradient norm. Default is 1.0
            experiment_name (str, optional): Experiment name. Default is default-experiment.
            iteration_num (int, optional): Iteration number. Defaults to 1.
            total_num_of_iterations (int, optional): Total number of iterations. Defaults to 1
            device_id (int, optional): Device id to run training on. Default to None.
            distributed (bool, optional): Whether to run distributed training. Default to False.

        Attributes:
            model_: Huggingface bert-base (default) pretrained model
            optimizer_: Stochastic gradient descent
            train_loader_: Training DataLoader
            test_loader_: Testing DataLoader
            labelToId_: label to ID mapping
            idToLabel_: ID to label mapping
            metric_: Evaluation metrics
            device_: Either cpu or gpu
        """

        # training params
        self._lr = lr
        self._epochs = epochs
        self._batch_size = batch_size
        self._experiment_name = experiment_name
        self._iteration_num = iteration_num
        self._model_path = model_path
        self._distributed = distributed

        # device
        self.device_ = (
            torch.device(
                torch.device("cuda", device_id) if torch.cuda.is_available() else "cpu"
            )
            if device_id is not None
            else torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        )
        logger.info(f"Using device: {self.device_}")

        if self._distributed:
            self._rank = device_id
            logger.info(f"Rank: {self._rank}")
        else:
            self._rank = None

        # tokenizer
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

        # dataset and data loader
        data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
        train_dataset, test_dataset = self.load_dataset(train_data_dir, test_data_dir)

        if self._distributed:
            logger.info("Setting up distributed samplers.")
            self.train_sampler_ = DistributedSampler(self.train_dataset_)
            self.test_sampler_ = DistributedSampler(self.test_dataset_)
        else:
            self.train_sampler_ = None
            self.test_sampler_ = None

        self.train_loader_ = DataLoader(
            train_dataset,
            shuffle=True,
            collate_fn=data_collator,
            batch_size=self._batch_size,
            sampler=self.train_sampler_,
        )
        self.test_loader_ = DataLoader(
            test_dataset,
            collate_fn=data_collator,
            batch_size=self._batch_size,
            sampler=self.train_sampler_,
        )

        logger.info(f"Train loader steps: {len(self.train_loader_)}")
        logger.info(f"Test loader steps: {len(self.test_loader_)}")

        # training params
        self.labelToId_ = pd.read_json("./examples/components/NER/traininsilo/labels.json", typ="series").to_dict()
        self.idToLabel_ = {val: key for key, val in self.labelToId_.items()}
        self.model_ = AutoModelForTokenClassification.from_pretrained(
            model_name,
            id2label=self.idToLabel_,
            label2id=self.labelToId_,
        )
        self.model_.train()
        if self._distributed:
            self.model_ = DDP(
                self.model_,
                device_ids=[self._rank] if self._rank is not None else None,
                output_device=self._rank,
            )
        self.model_.to(self.device_)
        self.metric_ = evaluate.load("seqeval")

        # DP
        logger.info(f"DP: {dp}")
        if dp:
            if not ModuleValidator.is_valid(self.model_):
                self.model_ = ModuleValidator.fix(self.model_)

        self.optimizer_ = AdamW(self.model_.parameters(), lr=2e-5)

        if dp:
            privacy_engine = PrivacyEngine(secure_mode=False)
            """secure_mode: Set to True if cryptographically strong DP guarantee is
            required. secure_mode=True uses secure random number generator for
            noise and shuffling (as opposed to pseudo-rng in vanilla PyTorch) and
            prevents certain floating-point arithmetic-based attacks.
            See :meth:~opacus.optimizers.optimizer._generate_noise for details.
            When set to True requires torchcsprng to be installed"""
            (
                self.model_,
                self.optimizer_,
                self.train_loader_,
            ) = privacy_engine.make_private_with_epsilon(
                module=self.model_,
                optimizer=self.optimizer_,
                data_loader=self.train_loader_,
                epochs=total_num_of_iterations * epochs,
                target_epsilon=dp_target_epsilon,
                target_delta=dp_target_delta,
                max_grad_norm=dp_max_grad_norm,
                batch_first=False,
            )

            """
            You can also obtain their counterparts by passing the noise multiplier. 
            Please refer to the following function.
            privacy_engine.make_private(
                module=self.model_,
                optimizer=self.optimizer_,
                data_loader=self.train_loader_,
                noise_multiplier=dp_noise_multiplier,
                max_grad_norm=dp_max_grad_norm,
            )
            """
            logger.info(
                f"Target epsilon: {dp_target_epsilon}, delta: {dp_target_delta} and noise multiplier: {self.optimizer_.noise_multiplier}"
            )

    def load_dataset(self, train_data_dir, test_data_dir):
        """Load dataset from {train_data_dir} and {test_data_dir}

        Args:
            train_data_dir(str, optional): Training data directory path
            test_data_dir(str, optional): Testing data directory path

        Returns:
            Train dataset
            Test datset
        """
        logger.info(f"Train data dir: {train_data_dir}, Test data dir: {test_data_dir}")
        train_dataset = load_from_disk(train_data_dir)
        test_dataset = load_from_disk(test_data_dir)

        return train_dataset, test_dataset

    def log_params(self, client, run_id):
        """Log parameters to the mlflow metrics

        Args:
            client (MlflowClient): Mlflow Client where metrics will be logged
            run_id (str): Run ID

        Returns:
            None
        """
        if run_id:
            client.log_param(
                run_id=run_id,
                key=f"learning_rate {self._experiment_name}",
                value=self._lr,
            )
            client.log_param(
                run_id=run_id, key=f"epochs {self._experiment_name}", value=self._epochs
            )
            client.log_param(
                run_id=run_id,
                key=f"batch_size {self._experiment_name}",
                value=self._batch_size,
            )
            client.log_param(
                run_id=run_id,
                key=f"optimizer {self._experiment_name}",
                value=self.optimizer_.__class__.__name__,
            )

    def log_metrics(self, client, run_id, key, value, pipeline_level=False):
        """Log mlflow metrics

        Args:
            client (MlflowClient): Mlflow Client where metrics will be logged
            run_id (str): Run ID
            key (str): metrics x-axis label
            value (float): metrics y-axis values
            pipeline_level (bool): whether the metrics is logged at the pipeline or the job level

        Returns:
            None
        """
        if run_id:
            if pipeline_level:
                client.log_metric(
                    run_id=run_id,
                    key=f"{self._experiment_name}/{key}",
                    value=value,
                )
            else:
                client.log_metric(
                    run_id=run_id,
                    key=f"iteration_{self._iteration_num}/{self._experiment_name}/{key}",
                    value=value,
                )

    def compute_metrics(self, eval_preds):
        """Compute evaluation results for given predictions

        Args:
            eval_preds (tuple): Contains logits and labels

        Returns:
            dict: Contains precision, recall, f1 and accuracy.
        """

        logits, labels = eval_preds
        predictions = np.argmax(logits, axis=-1)

        # Remove ignored index (special tokens) and convert to labels
        true_labels = [
            [self.idToLabel_[l] for l in label if l != -100] for label in labels
        ]
        true_predictions = [
            [self.idToLabel_[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        all_metrics = self.metric_.compute(
            predictions=true_predictions, references=true_labels
        )
        return {
            "precision": all_metrics["overall_precision"],
            "recall": all_metrics["overall_recall"],
            "f1": all_metrics["overall_f1"],
            "accuracy": all_metrics["overall_accuracy"],
        }

    def postprocess(self, predictions, labels):
        """Post-process predictions- remove padding

        Args:
            predictions (Tensor): Predicted labels
            labels (Tensor): Actual labels

        Returns:
            list[list]: Actual/True labels after removing padded tokens
            list[list]: Predicted labels after removing padded tokens
        """

        predictions = predictions.detach().cpu().clone().numpy()
        labels = labels.detach().cpu().clone().numpy()

        # Remove ignored index (special tokens) and convert to labels
        true_labels = [
            [self.idToLabel_[l] for l in label if l != -100] for label in labels
        ]
        true_predictions = [
            [self.idToLabel_[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        return true_labels, true_predictions

    def local_train(self, checkpoint):
        """Perform local training for a given number of epochs

        Args:
            checkpoint: Previous model checkpoint from where training has to be started.

        Returns:
            None
        """

        if checkpoint:
            if self._distributed:
                # DDP comes with "module." prefix: https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html
                self.model_.module.load_state_dict(
                    torch.load(checkpoint + "/model.pt", map_location=self.device_)
                )
            else:
                self.model_.load_state_dict(
                    torch.load(checkpoint + "/model.pt", map_location=self.device_)
                )

        with mlflow.start_run() as mlflow_run:
            # get Mlflow client and root run id
            mlflow_client = mlflow.tracking.client.MlflowClient()
            logger.debug(f"Root runId: {mlflow_run.data.tags.get('mlflow.rootRunId')}")
            root_run_id = mlflow_run.data.tags.get("mlflow.rootRunId")

            # log params
            self.log_params(mlflow_client, root_run_id)

            num_update_steps_per_epoch = len(self.train_loader_)
            num_training_steps = self._epochs * num_update_steps_per_epoch

            lr_scheduler = get_scheduler(
                "linear",
                optimizer=self.optimizer_,
                num_warmup_steps=0,
                num_training_steps=num_training_steps,
            )

            progress_bar = tqdm(range(num_training_steps))

            for epoch in range(self._epochs):

                running_loss = 0.0
                num_of_batches_before_logging = 100
                # Training
                self.model_.train()
                for i, batch in enumerate(self.train_loader_):
                    batch = {
                        key: value.to(self.device_) for key, value in batch.items()
                    }
                    outputs = self.model_(**batch)
                    loss = outputs.loss
                    loss.backward()
                    self.optimizer_.step()
                    lr_scheduler.step()
                    self.optimizer_.zero_grad()
                    progress_bar.update(1)

                    # calculate metric
                    true_predictions, true_labels = self.postprocess(
                        outputs.logits.argmax(dim=-1), batch["labels"]
                    )
                    self.metric_.add_batch(
                        predictions=true_predictions, references=true_labels
                    )

                    running_loss += float(loss) / len(batch)

                    del outputs
                    del batch

                    if i != 0 and i % num_of_batches_before_logging == 0:
                        training_loss = running_loss / num_of_batches_before_logging
                        logger.info(
                            f"Epoch: {epoch}/{self._epochs}, Iteration: {i}, Training Loss: {training_loss}"
                        )

                        if not self._distributed or self._rank == 0:
                            # log train loss
                            self.log_metrics(
                                mlflow_client,
                                root_run_id,
                                "Train Loss",
                                training_loss,
                            )

                            # log evaluation metrics
                            results = self.metric_.compute()
                            for key in ["precision", "recall", "f1", "accuracy"]:
                                self.log_metrics(
                                    mlflow_client,
                                    root_run_id,
                                    f"Train {key}",
                                    results[f"overall_{key}"],
                                )

                        running_loss = 0.0

                test_loss, metric_results = self.test()

                # log test loss for each epoch
                if not self._distributed or self._rank == 0:
                    self.log_metrics(mlflow_client, root_run_id, "Test Loss", test_loss)
                logger.info(f"Epoch: {epoch}, Test Loss: {test_loss}")

                # log test metric for each epoch
                for key in ["precision", "recall", "f1", "accuracy"]:
                    logger.info(
                        f"Epoch: {epoch}: Test {key} is {metric_results[f'overall_{key}']}"
                    )
                    if not self._distributed or self._rank == 0:
                        self.log_metrics(
                            mlflow_client,
                            root_run_id,
                            f"Test {key}",
                            metric_results[f"overall_{key}"],
                        )

            # log metrics for each FL iteration
            if not self._distributed or self._rank == 0:
                self.log_metrics(
                    mlflow_client,
                    root_run_id,
                    "Train Loss",
                    training_loss,
                    pipeline_level=True,
                )
                self.log_metrics(
                    mlflow_client,
                    root_run_id,
                    "Test Loss",
                    test_loss,
                    pipeline_level=True,
                )

                for key in ["precision", "recall", "f1", "accuracy"]:
                    self.log_metrics(
                        mlflow_client,
                        root_run_id,
                        f"Test {key}",
                        metric_results[f"overall_{key}"],
                        pipeline_level=True,
                    )

    def test(self):
        """Test the trained model and report test loss and metrics"""
        self.model_.eval()
        test_loss = 0
        with torch.no_grad():
            for batch in self.test_loader_:
                batch = {key: value.to(self.device_) for key, value in batch.items()}
                outputs = self.model_(**batch)
                predictions = outputs.logits.argmax(dim=-1)
                labels = batch["labels"]
                test_loss += float(outputs.loss)

                del outputs
                del batch

                true_predictions, true_labels = self.postprocess(predictions, labels)
                self.metric_.add_batch(
                    predictions=true_predictions, references=true_labels
                )

        results = self.metric_.compute()
        test_loss /= len(self.test_loader_)

        return test_loss, results

    def execute(self, checkpoint=None):
        """Bundle steps to perform local training, model testing and finally saving the model.

        Args:
            checkpoint: Previous model checkpoint from where training has to be started.
        """
        logger.debug("Start training")
        self.local_train(checkpoint)

        if not self._distributed:
            logger.debug("Save model")
            torch.save(self.model_.state_dict(), self._model_path)
            logger.info(f"Model saved to {self._model_path}")
        elif self._rank == 0:
            # DDP comes with "module." prefix: https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html
            logger.debug("Save model")
            torch.save(self.model_.module.state_dict(), self._model_path)
            logger.info(f"Model saved to {self._model_path}")

In [2]:
class NERTrainer:
    def __init__(
        self,
        tokenizer_name,
        model_name,
        train_data_dir="./",
        test_data_dir="./",
        model_path=None,
        lr=0.01,
        epochs=1,
        batch_size=64,
        dp=False,
        dp_target_epsilon=50.0,
        dp_target_delta=1e-5,
        dp_max_grad_norm=1.0,
        experiment_name="default-experiment",
        iteration_num=1,
        total_num_of_iterations=1,
        device_id=None,
        distributed=False,
    ):
        """NER Trainer trains BERT-base model (default) on the MultiNERD dataset.

        Args:
            tokenizer_name(str): Tokenizer name.
            model_name(str): Model name.
            train_data_dir(str, optional): Training data directory path.
            test_data_dir(str, optional): Testing data directory path.
            model_path (str, optional): Model path. Defaults to None.
            lr (float, optional): Learning rate. Defaults to 0.01.
            epochs (int, optional): number of epochs. Defaults to 1.
            batch_size (int, optional): DataLoader batch size. Defaults to 64.
            dp (bool, optional): Differential Privacy. Default is False
            dp_target_epsilon (float, optional): DP target epsilon. Default is 50.0
            dp_target_delta (float, optional): DP target delta. Default is 1e-5
            dp_max_grad_norm (float, optional): DP max gradient norm. Default is 1.0
            experiment_name (str, optional): Experiment name. Default is default-experiment.
            iteration_num (int, optional): Iteration number. Defaults to 1.
            total_num_of_iterations (int, optional): Total number of iterations. Defaults to 1
            device_id (int, optional): Device id to run training on. Default to None.
            distributed (bool, optional): Whether to run distributed training. Default to False.

        Attributes:
            model_: Huggingface bert-base (default) pretrained model
            optimizer_: Stochastic gradient descent
            train_loader_: Training DataLoader
            test_loader_: Testing DataLoader
            labelToId_: label to ID mapping
            idToLabel_: ID to label mapping
            metric_: Evaluation metrics
            device_: Either cpu or gpu
        """

        # training params
        self._lr = lr
        self._epochs = epochs
        self._batch_size = 1
        self._experiment_name = experiment_name
        self._iteration_num = iteration_num
        self._model_path = model_path
        self._distributed = distributed

        # device
        self.device_ = (
            torch.device(
                torch.device("cuda", device_id) if torch.cuda.is_available() else "cpu"
            )
            if device_id is not None
            else torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        )
        print(f"Using device: {self.device_}")

        if self._distributed:
            self._rank = device_id
            print(f"Rank: {self._rank}")
        else:
            self._rank = None

        # tokenizer
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

        # dataset and data loader
        data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
        train_dataset, test_dataset = self.load_dataset(train_data_dir, test_data_dir)

        if self._distributed:
            print("Setting up distributed samplers.")
            self.train_sampler_ = DistributedSampler(self.train_dataset_)
            self.test_sampler_ = DistributedSampler(self.test_dataset_)
        else:
            self.train_sampler_ = None
            self.test_sampler_ = None

        self.train_loader_ = DataLoader(
            train_dataset,
            shuffle=True,
            collate_fn=data_collator,
            batch_size=self._batch_size,
            sampler=self.train_sampler_,
        )
        self.test_loader_ = DataLoader(
            test_dataset,
            collate_fn=data_collator,
            batch_size=self._batch_size,
            sampler=self.train_sampler_,
        )

        print(f"Train loader steps: {len(self.train_loader_)}")
        print(f"Test loader steps: {len(self.test_loader_)}")

        # training params
        self.labelToId_ = pd.read_json("./examples/components/NER/traininsilo/labels.json", typ="series").to_dict()
        self.idToLabel_ = {val: key for key, val in self.labelToId_.items()}
        self.model_ = AutoModelForTokenClassification.from_pretrained(
            model_name,
            id2label=self.idToLabel_,
            label2id=self.labelToId_,
        )
        # trainable_layers = [self.model_.bert.encoder.layer[-1], self.model_.classifier]
        # trainable_params=0
        # for layer in trainable_layers:
        #     for p in layer.parameters():
        #         p.requires_grad = True
        #         trainable_params += p.numel()
        # print(trainable_params)
        if self._distributed:
            self.model_ = DDP(
                self.model_,
                device_ids=[self._rank] if self._rank is not None else None,
                output_device=self._rank,
            )
        self.model_.to(self.device_)
        self.metric_ = evaluate.load("seqeval")
        self.model_.train()
        # DP
        print(f"DP: {dp}")
        if dp:
            if not ModuleValidator.is_valid(self.model_):
                self.model_ = ModuleValidator.fix(self.model_)

        self.optimizer_ = AdamW(self.model_.parameters(), lr=2e-5)
        print([(n, p.shape) for (n, p) in self.model_.named_parameters()])
        if dp:
            privacy_engine = PrivacyEngine(secure_mode=False)
            """secure_mode: Set to True if cryptographically strong DP guarantee is
            required. secure_mode=True uses secure random number generator for
            noise and shuffling (as opposed to pseudo-rng in vanilla PyTorch) and
            prevents certain floating-point arithmetic-based attacks.
            See :meth:~opacus.optimizers.optimizer._generate_noise for details.
            When set to True requires torchcsprng to be installed"""
            (
                self.model_,
                self.optimizer_,
                self.train_loader_,
            ) = privacy_engine.make_private_with_epsilon(
                module=self.model_,
                optimizer=self.optimizer_,
                data_loader=self.train_loader_,
                epochs=total_num_of_iterations * epochs,
                target_epsilon=dp_target_epsilon,
                target_delta=dp_target_delta,
                max_grad_norm=dp_max_grad_norm,
                batch_first=False,
            )
            # print([(n, p.grad_sample.shape) for (n, p) in self.model_.named_parameters()])

            """
            You can also obtain their counterparts by passing the noise multiplier. 
            Please refer to the following function.
            privacy_engine.make_private(
                module=self.model_,
                optimizer=self.optimizer_,
                data_loader=self.train_loader_,
                noise_multiplier=dp_noise_multiplier,
                max_grad_norm=dp_max_grad_norm,
            )
            """
            print(
                f"Target epsilon: {dp_target_epsilon}, delta: {dp_target_delta} and noise multiplier: {self.optimizer_.noise_multiplier}"
            )

    def load_dataset(self, train_data_dir, test_data_dir):
        """Load dataset from {train_data_dir} and {test_data_dir}

        Args:
            train_data_dir(str, optional): Training data directory path
            test_data_dir(str, optional): Testing data directory path

        Returns:
            Train dataset
            Test datset
        """
        print(f"Train data dir: {train_data_dir}, Test data dir: {test_data_dir}")
        train_dataset = load_from_disk(train_data_dir)
        test_dataset = load_from_disk(test_data_dir)

        return train_dataset, test_dataset

    def log_params(self, client, run_id):
        """Log parameters to the mlflow metrics

        Args:
            client (MlflowClient): Mlflow Client where metrics will be logged
            run_id (str): Run ID

        Returns:
            None
        """
        if run_id:
            client.log_param(
                run_id=run_id,
                key=f"learning_rate {self._experiment_name}",
                value=self._lr,
            )
            client.log_param(
                run_id=run_id, key=f"epochs {self._experiment_name}", value=self._epochs
            )
            client.log_param(
                run_id=run_id,
                key=f"batch_size {self._experiment_name}",
                value=self._batch_size,
            )
            client.log_param(
                run_id=run_id,
                key=f"optimizer {self._experiment_name}",
                value=self.optimizer_.__class__.__name__,
            )

    def log_metrics(self, client, run_id, key, value, pipeline_level=False):
        """Log mlflow metrics

        Args:
            client (MlflowClient): Mlflow Client where metrics will be logged
            run_id (str): Run ID
            key (str): metrics x-axis label
            value (float): metrics y-axis values
            pipeline_level (bool): whether the metrics is logged at the pipeline or the job level

        Returns:
            None
        """
        if run_id:
            if pipeline_level:
                client.log_metric(
                    run_id=run_id,
                    key=f"{self._experiment_name}/{key}",
                    value=value,
                )
            else:
                client.log_metric(
                    run_id=run_id,
                    key=f"iteration_{self._iteration_num}/{self._experiment_name}/{key}",
                    value=value,
                )

    def compute_metrics(self, eval_preds):
        """Compute evaluation results for given predictions

        Args:
            eval_preds (tuple): Contains logits and labels

        Returns:
            dict: Contains precision, recall, f1 and accuracy.
        """

        logits, labels = eval_preds
        predictions = np.argmax(logits, axis=-1)

        # Remove ignored index (special tokens) and convert to labels
        true_labels = [
            [self.idToLabel_[l] for l in label if l != -100] for label in labels
        ]
        true_predictions = [
            [self.idToLabel_[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        all_metrics = self.metric_.compute(
            predictions=true_predictions, references=true_labels
        )
        return {
            "precision": all_metrics["overall_precision"],
            "recall": all_metrics["overall_recall"],
            "f1": all_metrics["overall_f1"],
            "accuracy": all_metrics["overall_accuracy"],
        }

    def postprocess(self, predictions, labels):
        """Post-process predictions- remove padding

        Args:
            predictions (Tensor): Predicted labels
            labels (Tensor): Actual labels

        Returns:
            list[list]: Actual/True labels after removing padded tokens
            list[list]: Predicted labels after removing padded tokens
        """

        predictions = predictions.detach().cpu().clone().numpy()
        labels = labels.detach().cpu().clone().numpy()

        # Remove ignored index (special tokens) and convert to labels
        true_labels = [
            [self.idToLabel_[l] for l in label if l != -100] for label in labels
        ]
        true_predictions = [
            [self.idToLabel_[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        return true_labels, true_predictions

    def local_train(self, checkpoint):
        """Perform local training for a given number of epochs

        Args:
            checkpoint: Previous model checkpoint from where training has to be started.

        Returns:
            None
        """

        if checkpoint:
            if self._distributed:
                # DDP comes with "module." prefix: https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html
                self.model_.module.load_state_dict(
                    torch.load(checkpoint + "/model.pt", map_location=self.device_)
                )
            else:
                self.model_.load_state_dict(
                    torch.load(checkpoint + "/model.pt", map_location=self.device_)
                )

        with mlflow.start_run() as mlflow_run:
            # get Mlflow client and root run id
            mlflow_client = mlflow.tracking.client.MlflowClient()
            logger.debug(f"Root runId: {mlflow_run.data.tags.get('mlflow.rootRunId')}")
            root_run_id = mlflow_run.data.tags.get("mlflow.rootRunId")

            # log params
            self.log_params(mlflow_client, root_run_id)

            num_update_steps_per_epoch = len(self.train_loader_)
            num_training_steps = self._epochs * num_update_steps_per_epoch

            lr_scheduler = get_scheduler(
                "linear",
                optimizer=self.optimizer_,
                num_warmup_steps=0,
                num_training_steps=num_training_steps,
            )

            progress_bar = tqdm(range(num_training_steps))

            for epoch in range(self._epochs):

                running_loss = 0.0
                num_of_batches_before_logging = 100
                # Training
                self.model_.train()
                for i, batch in enumerate(self.train_loader_):
                    try:
                        batch = {
                            key: value.to(self.device_) for key, value in batch.items()
                        }
                        
                        outputs = self.model_(**batch)
                        loss = outputs.loss
                        loss.backward()
                        self.optimizer_.step()
                        lr_scheduler.step()
                        self.optimizer_.zero_grad()
                        progress_bar.update(1)
                        print([(n, p.grad_sample.shape) for (n, p) in self.model_.named_parameters()])
                    except Exception as e:
                        print(e)
                        print([(n, p.grad_sample.shape) for (n, p) in self.model_.named_parameters()])
                        print([(n, p.shape) for (n, p) in self.model_.named_parameters()])
                        print(batch['input_ids'].shape)
                        print(batch['labels'].shape)
                        print(batch)

                    # calculate metric
                    true_predictions, true_labels = self.postprocess(
                        outputs.logits.argmax(dim=-1), batch["labels"]
                    )
                    self.metric_.add_batch(
                        predictions=true_predictions, references=true_labels
                    )

                    running_loss += float(loss) / len(batch)

                    del outputs
                    del batch

                    if i != 0 and i % num_of_batches_before_logging == 0:
                        training_loss = running_loss / num_of_batches_before_logging
                        print(
                            f"Epoch: {epoch}/{self._epochs}, Iteration: {i}, Training Loss: {training_loss}"
                        )

                        if not self._distributed or self._rank == 0:
                            # log train loss
                            self.log_metrics(
                                mlflow_client,
                                root_run_id,
                                "Train Loss",
                                training_loss,
                            )

                            # log evaluation metrics
                            results = self.metric_.compute()
                            for key in ["precision", "recall", "f1", "accuracy"]:
                                self.log_metrics(
                                    mlflow_client,
                                    root_run_id,
                                    f"Train {key}",
                                    results[f"overall_{key}"],
                                )

                        running_loss = 0.0

                test_loss, metric_results = self.test()

                # log test loss for each epoch
                if not self._distributed or self._rank == 0:
                    self.log_metrics(mlflow_client, root_run_id, "Test Loss", test_loss)
                print(f"Epoch: {epoch}, Test Loss: {test_loss}")

                # log test metric for each epoch
                for key in ["precision", "recall", "f1", "accuracy"]:
                    print(
                        f"Epoch: {epoch}: Test {key} is {metric_results[f'overall_{key}']}"
                    )
                    if not self._distributed or self._rank == 0:
                        self.log_metrics(
                            mlflow_client,
                            root_run_id,
                            f"Test {key}",
                            metric_results[f"overall_{key}"],
                        )

            # log metrics for each FL iteration
            if not self._distributed or self._rank == 0:
                self.log_metrics(
                    mlflow_client,
                    root_run_id,
                    "Train Loss",
                    training_loss,
                    pipeline_level=True,
                )
                self.log_metrics(
                    mlflow_client,
                    root_run_id,
                    "Test Loss",
                    test_loss,
                    pipeline_level=True,
                )

                for key in ["precision", "recall", "f1", "accuracy"]:
                    self.log_metrics(
                        mlflow_client,
                        root_run_id,
                        f"Test {key}",
                        metric_results[f"overall_{key}"],
                        pipeline_level=True,
                    )

    def test(self):
        """Test the trained model and report test loss and metrics"""
        self.model_.eval()
        test_loss = 0
        with torch.no_grad():
            for batch in self.test_loader_:
                batch = {key: value.to(self.device_) for key, value in batch.items()}
                outputs = self.model_(**batch)
                predictions = outputs.logits.argmax(dim=-1)
                labels = batch["labels"]
                test_loss += float(outputs.loss)

                del outputs
                del batch

                true_predictions, true_labels = self.postprocess(predictions, labels)
                self.metric_.add_batch(
                    predictions=true_predictions, references=true_labels
                )

        results = self.metric_.compute()
        test_loss /= len(self.test_loader_)

        return test_loss, results

    def execute(self, checkpoint=None):
        """Bundle steps to perform local training, model testing and finally saving the model.

        Args:
            checkpoint: Previous model checkpoint from where training has to be started.
        """
        logger.debug("Start training")
        print("ok")
        self.local_train(checkpoint)

        if not self._distributed:
            logger.debug("Save model")
            torch.save(self.model_.state_dict(), self._model_path)
            print(f"Model saved to {self._model_path}")
        elif self._rank == 0:
            # DDP comes with "module." prefix: https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html
            logger.debug("Save model")
            torch.save(self.model_.module.state_dict(), self._model_path)
            print(f"Model saved to {self._model_path}")

In [4]:
# Set logging to sys.out
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
log_format = logging.Formatter("[%(asctime)s] [%(levelname)s] - %(message)s")
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
handler.setFormatter(log_format)
logger.addHandler(handler)

In [4]:
print([(n, p.shape) for (n, p) in trainer.model_.named_parameters()])

NameError: name 'trainer' is not defined

In [7]:
trainer = NERTrainer(
    tokenizer_name="bert-base-cased",
    model_name="bert-base-cased",
    train_data_dir="./ner_error_debug/preprocess_train/",
    test_data_dir="./ner_error_debug/preprocess_test/",
    dp=True,
)

[2023-01-26 23:35:58,588] [INFO] - Using device: cpu


01/26/2023 23:35:58:INFO:Using device: cpu


[2023-01-26 23:35:59,085] [INFO] - Train data dir: ./ner_error_debug/preprocess_train/, Test data dir: ./ner_error_debug/preprocess_test/


01/26/2023 23:35:59:INFO:Train data dir: ./ner_error_debug/preprocess_train/, Test data dir: ./ner_error_debug/preprocess_test/


[2023-01-26 23:35:59,120] [INFO] - Train loader steps: 2309


...

01/26/2023 23:35:59:INFO:Test loader steps: 257
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initia

[2023-01-26 23:36:01,607] [INFO] - DP: True


01/26/2023 23:36:01:INFO:DP: True
c:\Users\Amit Garg\miniconda3\envs\fl_experiment_conda_env\lib\site-packages\opacus\privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
c:\Users\Amit Garg\miniconda3\envs\fl_experiment_conda_env\lib\site-packages\opacus\accountants\analysis\rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
c:\Users\Amit Garg\miniconda3\envs\fl_experiment_conda_env\lib\site-packages\opacus\accountants\analysis\prv\prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)
c:\Users\Amit Garg\miniconda3\envs\fl_experiment_conda_env\lib\site-packages\opacus\accountants\analysis\rdp.py:332: UserWarning: Optimal or

[2023-01-26 23:36:43,433] [INFO] - Target epsilon: 50.0, delta: 1e-05 and noise multiplier: 0.20936965942382812


01/26/2023 23:36:43:INFO:Target epsilon: 50.0, delta: 1e-05 and noise multiplier: 0.20936965942382812


In [8]:
trainer.execute()

[2023-01-26 23:36:49,207] [DEBUG] - Start training


01/26/2023 23:36:49:DEBUG:Start training


[2023-01-26 23:36:49,617] [DEBUG] - Root runId: None


01/26/2023 23:36:49:DEBUG:Root runId: None
  0%|          | 0/2309 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\Amit Garg\miniconda3\envs\fl_experiment_conda_env\lib\site-packages\torch\nn\modules\module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
c:\Users\Amit Garg\miniconda3\envs\fl_experiment_conda_env\lib\site-packages\torch\nn\modules\module.py:1082: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Ten

KeyboardInterrupt: 

In [8]:
trainer.train_loader_

In [7]:
for i in trainer.train_loader_:
    print(i)
    exit

TypeError: zeros() received an invalid combination of arguments - got (tuple, dtype=type), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of SymInts size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [16]:
trainer2 = NERTrainer(
    tokenizer_name="bert-base-cased",
    model_name="bert-base-cased",
    train_data_dir="./ner_error_debug/preprocess_train/",
    test_data_dir="./ner_error_debug/preprocess_test/",
    dp=False,
)

Using device: cpu
Train data dir: ./ner_error_debug/preprocess_train/, Test data dir: ./ner_error_debug/preprocess_test/
Train loader steps: 2309
Test loader steps: 257


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

DP: False


In [17]:
trainer2.execute()

[2023-01-26 16:03:40,360] [DEBUG] - Start training


01/26/2023 16:03:40:DEBUG:Start training


ok
[2023-01-26 16:03:40,459] [DEBUG] - Root runId: None


01/26/2023 16:03:40:DEBUG:Root runId: None
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


'Parameter' object has no attribute 'grad_sample'


AttributeError: 'Parameter' object has no attribute 'grad_sample'

In [19]:
trainer.model_

GradSampleModule(BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

In [11]:
labelToId_ = pd.read_json("./examples/components/NER/traininsilo/labels.json", typ="series").to_dict()
idToLabel_ = {val: key for key, val in labelToId_.items()}
tmp = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    id2label=idToLabel_,
    label2id=labelToId_,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [23]:
trainable_layers = [tmp.bert.encoder.layer[-1], tmp.classifier]

In [24]:
trainable_params=0
for layer in trainable_layers:
    for p in layer.parameters():
        p.requires_grad = True
        trainable_params += p.numel()

In [25]:
trainable_params

7114787

### Homomorphic Encryption

In [3]:
import os
import argparse
import logging
import sys
import glob

import torch
import tenseal as ts
import tenseal.sealapi as sealapi
import base64

c:\Users\gargamit\Miniconda3\envs\fl_experiment_conda_env_latest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from torchvision import models, datasets, transforms
from torch import nn

model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(
    1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

c:\Users\gargamit\Miniconda3\envs\fl_experiment_conda_env_latest\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\gargamit\Miniconda3\envs\fl_experiment_conda_env_latest\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
model.load_state_dict(
                    torch.load("C:/Users/gargamit/Downloads/mnist_fl_model.pt", map_location=torch.device('cpu'))
                )

<All keys matched successfully>

In [31]:
model.state_dict().keys()

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer2.0.conv1.weight', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.bn1.num_batches_tracked', 'layer2.0.conv2.weight', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', '

In [32]:
tp = {}
for i in model.state_dict().keys():
    tp[i] = model.state_dict()[i]

In [23]:
from omegaconf import OmegaConf
# tmp = "C:\Users\Amit Garg\Downloads\MLTable"
YAML_CONFIG = OmegaConf.load(r"C:\Users\Amit Garg\Downloads\MLTable")
# with open(r"C:\Users\Amit Garg\Downloads\MLTable", "r") as f:
#     print(f.readlines())

In [8]:
YAML_CONFIG

{'paths': [{'folder': 'azureml://subscriptions/48bbc269-ce89-4f6f-9a12-c6f91fcb772d/resourcegroups/amitgarg-fldev-rg/workspaces/aml-flhe/datastores/datastore_orchestrator/paths/federated_learning/model/5b8d46ed-504e-4187-9c8f-7612d24ead8d/'}, {'folder': 'azureml://subscriptions/48bbc269-ce89-4f6f-9a12-c6f91fcb772d/resourcegroups/amitgarg-fldev-rg/workspaces/aml-flhe/datastores/datastore_orchestrator/paths/federated_learning/model/8e7df04b-b93f-423a-bd74-5ed910a29b47/'}, {'folder': 'azureml://subscriptions/48bbc269-ce89-4f6f-9a12-c6f91fcb772d/resourcegroups/amitgarg-fldev-rg/workspaces/aml-flhe/datastores/datastore_orchestrator/paths/federated_learning/model/6cffd8bd-8807-4107-ab94-15eabf8377b7/'}]}

In [12]:
for path in YAML_CONFIG.paths:
    print(path['folder'])

azureml://subscriptions/48bbc269-ce89-4f6f-9a12-c6f91fcb772d/resourcegroups/amitgarg-fldev-rg/workspaces/aml-flhe/datastores/datastore_orchestrator/paths/federated_learning/model/5b8d46ed-504e-4187-9c8f-7612d24ead8d/
azureml://subscriptions/48bbc269-ce89-4f6f-9a12-c6f91fcb772d/resourcegroups/amitgarg-fldev-rg/workspaces/aml-flhe/datastores/datastore_orchestrator/paths/federated_learning/model/8e7df04b-b93f-423a-bd74-5ed910a29b47/
azureml://subscriptions/48bbc269-ce89-4f6f-9a12-c6f91fcb772d/resourcegroups/amitgarg-fldev-rg/workspaces/aml-flhe/datastores/datastore_orchestrator/paths/federated_learning/model/6cffd8bd-8807-4107-ab94-15eabf8377b7/


In [24]:
tmp = {"asd":324}

In [27]:
print(tmp.get("aasdsd"))

None


In [28]:
import os

os.path.join("azureml://subscriptions/48bbc269-ce89-4f6f-9a12-c6f91fcb772d/resourcegroups/amitgarg-fldev-rg/workspaces/aml-flhe/datastores/datastore_orchestrator/paths/federated_learning/model/5b8d46ed-504e-4187-9c8f-7612d24ead8d/", ".pt")

'azureml://subscriptions/48bbc269-ce89-4f6f-9a12-c6f91fcb772d/resourcegroups/amitgarg-fldev-rg/workspaces/aml-flhe/datastores/datastore_orchestrator/paths/federated_learning/model/5b8d46ed-504e-4187-9c8f-7612d24ead8d/.pt'

In [32]:
import glob

glob.glob("C:/Users/", recursive=True)

['C:/Users/']

In [38]:
for i in os.listdir(r"C:\Users\Amit Garg\Downloads\packages (1)\packages\azure-ai-ml"):
    if i.endswith(".whl"):
        print(i)

azure_ai_ml-1.5.0-py3-none-any.whl


In [40]:
import multiprocessing

In [41]:
multiprocessing.cpu_count()

24

In [43]:
import os
for subdir, dirs, files in os.walk(r"C:\Users\Amit Garg\OneDrive - Microsoft\Documents\fl_sdk_test"):
  for file in files:
    if file.endswith(".txt"):
      print(os.path.join(subdir, file))

C:\Users\Amit Garg\OneDrive - Microsoft\Documents\fl_sdk_test\findings.txt.txt
C:\Users\Amit Garg\OneDrive - Microsoft\Documents\fl_sdk_test\datastores\silo_datastore1\first.txt
C:\Users\Amit Garg\OneDrive - Microsoft\Documents\fl_sdk_test\datastores\silo_datastore2\second.txt
C:\Users\Amit Garg\OneDrive - Microsoft\Documents\fl_sdk_test\datastores\silo_datastore3\third.txt
